# IS319 - Deep Learning

## TP3 - Recurrent neural networks

Credits: Andrej Karpathy

The goal of this TP is to experiment with recurrent neural networks for a character-level language model to generate text that looks like training text data.

In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Text data preprocessing

Several text datasets are provided, feel free to experiment with different ones throughout the TP. At the beginning, use a small subset of a given dataset (for example use only 10k characters).

In [87]:
!tar -xvf text-datasets.tgz

tar: text-datasets.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [88]:
text_data_fname = 'baudelaire.txt'  # ~0.1m characters (French)
# text_data_fname = 'proust.txt'      # ~7.3m characters (French)
# text_data_fname = 'shakespeare.txt' # ~0.1m characters (English)
# text_data_fname = 'lotr.txt'        # ~2.5m characters (English)
# text_data_fname = 'doom.c'          # ~1m characters (C Code)
# text_data_fname = 'linux.c'         # ~11.5m characters (C code)

text_data = open(text_data_fname, 'r').read()
text_data = text_data[:10000] # use a small subset
print(f'Dataset `{text_data_fname}` contains {len(text_data)} characters.')
print('Excerpt of the dataset:')
print(text_data[:2000])

Dataset `baudelaire.txt` contains 10000 characters.
Excerpt of the dataset:
LES FLEURS DU MAL

par

CHARLES BAUDELAIRE


AU LECTEUR


La sottise, l'erreur, le péché, la lésine,
Occupent nos esprits et travaillent nos corps,
Et nous alimentons nos aimables remords,
Comme les mendiants nourrissent leur vermine.

Nos péchés sont têtus, nos repentirs sont lâches,
Nous nous faisons payer grassement nos aveux,
Et nous rentrons gaîment dans le chemin bourbeux,
Croyant par de vils pleurs laver toutes nos taches.

Sur l'oreiller du mal c'est Satan Trismégiste
Qui berce longuement notre esprit enchanté,
Et le riche métal de notre volonté
Est tout vaporisé par ce savant chimiste.

C'est le Diable qui tient les fils qui nous remuent!
Aux objets répugnants nous trouvons des appas;
Chaque jour vers l'Enfer nous descendons d'un pas,
Sans horreur, à travers des ténèbres qui puent.

Ainsi qu'un débauché pauvre qui baise et mange
Le sein martyrisé d'une antique catin,
Nous volons au passage un plaisir c

**(Question)** Create a character-level vocabulary for your text data. Create two dictionaries: `ctoi` mapping each character to an index, and the reverse `itoc` mapping each index to its corresponding character. Implement the functions to convert text to tensor and tensor to text using these mappings. Apply these functions to some text data.

In [106]:
# Create the vocabulary and the two mapping dictionaries
# YOUR CODE HERE
import numpy as np

idx = 0
ctoi = {}
itoc = {}

voc = set(text_data)

for elt in voc:
    ctoi[elt] = idx
    itoc[idx] = elt
    idx += 1

print(ctoi)
print(itoc)
print(len(ctoi))

# Implement the function converting text to tensor
def text_to_tensor(text, ctoi):
    # YOUR CODE HERE
    return torch.LongTensor(np.array([ctoi[c] for c in text]))
    #tensor = torch.zeros(len(text), 1, len(ctoi))
    #for i, letter in enumerate(text):
    #  tensor[i][0][ctoi[letter]] = 1
    #return tensor


# Implement the function converting tensor to text
def tensor_to_text(tensor, itoc):
    # YOUR CODE HERE
    return ''.join([itoc[elt.item()] for elt in tensor])#torch.argmax(tensor, dim=2)])

# Apply your functions to some text data
# YOUR CODE HERE
#raise NotImplementedError()
a = text_to_tensor(text_data[:10], ctoi)

print(a)
print(tensor_to_text(a, itoc))



{'x': 0, 'g': 1, 'b': 2, 'E': 3, 'R': 4, '\n': 5, 'j': 6, 'p': 7, 'ô': 8, 'I': 9, 'T': 10, 'S': 11, 'ê': 12, 'â': 13, 'W': 14, 'm': 15, '_': 16, '?': 17, 'F': 18, ' ': 19, 'h': 20, 'U': 21, "'": 22, 'D': 23, 'r': 24, 'n': 25, 'd': 26, 'f': 27, 'Q': 28, '!': 29, 'L': 30, 'ù': 31, 'û': 32, 'G': 33, 'P': 34, 'e': 35, 'ç': 36, 'l': 37, '.': 38, ',': 39, 'è': 40, 'É': 41, ';': 42, '«': 43, 'q': 44, 'M': 45, 'y': 46, 'V': 47, 'î': 48, 'u': 49, 'k': 50, ':': 51, 'J': 52, 'é': 53, 'O': 54, 'B': 55, 'A': 56, 'v': 57, 'c': 58, '»': 59, 'i': 60, 'o': 61, 'C': 62, 't': 63, 'à': 64, '-': 65, 'N': 66, 's': 67, 'H': 68, 'a': 69, 'z': 70}
{0: 'x', 1: 'g', 2: 'b', 3: 'E', 4: 'R', 5: '\n', 6: 'j', 7: 'p', 8: 'ô', 9: 'I', 10: 'T', 11: 'S', 12: 'ê', 13: 'â', 14: 'W', 15: 'm', 16: '_', 17: '?', 18: 'F', 19: ' ', 20: 'h', 21: 'U', 22: "'", 23: 'D', 24: 'r', 25: 'n', 26: 'd', 27: 'f', 28: 'Q', 29: '!', 30: 'L', 31: 'ù', 32: 'û', 33: 'G', 34: 'P', 35: 'e', 36: 'ç', 37: 'l', 38: '.', 39: ',', 40: 'è', 41: 'É',

## 2. Setup a character-level recurrent neural network

**(Question)** Setup a simple embedding layer with `nn.Embedding` to project character indices to `embedding_dim` dimensional vectors. Explain precisely how this layer works and what are its outputs for a given input sequence.

In [103]:
# YOUR CODE HERE
embedding = nn.Embedding(len(ctoi),5)

embedding(text_to_tensor(text_data[:10], ctoi))

tensor([[-1.0499, -2.5538, -0.8329,  0.2914, -1.7585],
        [-0.5466,  0.7686, -0.3067, -0.6744, -0.1041],
        [ 0.2995,  0.6213, -1.7441, -0.9082,  0.3287],
        [ 0.0716, -0.2112,  0.7629,  2.0637, -0.6587],
        [ 1.3853, -0.3015, -0.1482,  0.0467,  1.3081],
        [-1.0499, -2.5538, -0.8329,  0.2914, -1.7585],
        [-0.5466,  0.7686, -0.3067, -0.6744, -0.1041],
        [ 0.0631,  0.7991, -0.8591,  1.1461,  1.1999],
        [ 0.4302,  0.0691,  1.5564, -1.0611,  1.5152],
        [ 0.2995,  0.6213, -1.7441, -0.9082,  0.3287]],
       grad_fn=<EmbeddingBackward0>)

**(Question)** Setup a single-layer RNN with `nn.RNN` (without defining a custom class). Use `hidden_dim` size for hidden states. Explain precisely the outputs of this layer for a given input sequence.

In [91]:
# YOUR CODE HERE
rnn = nn.RNN(input_size=5, hidden_size=20)

YOUR ANSWER HERE

**(Question)** Create a simple RNN model with a custom `nn.Module` class. It should contain: an embedding layer, a single-layer RNN, and a dense output layer. For each character of the input sequence, the model should predict the probability of the next character. The forward method should return the probabilities for next characters and the corresponding hidden states.
After completing the class, create a model and apply the forward pass on some input text. Understand and explain the results.

*Note:* depending on how you implement the loss function later, it can be convenient to return logits instead of probabilities, i.e. raw values of the output layer before any activation function.

In [92]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        '''Initialize model parameters and layers.'''
        super().__init__()
        # YOUR CODE HERE
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers)
        self.dense = nn.Linear(hidden_dim, vocab_size)

    def forward(self, tensor_data, hidden_state=None):
        '''Apply the forward pass for some text data already converted to tensor.'''
        # YOUR CODE HERE
        embedding = self.embedding(tensor_data)
        output, hidden = self.rnn(embedding, hidden_state)
        logits = self.dense(output)
        return logits, hidden

# Initialize a model and apply the forward pass on some input text
# YOUR CODE HERE
charRNN = CharRNN(len(ctoi),5,20)
logits, _ = charRNN.forward(text_to_tensor(text_data[:1],ctoi))

print(text_data[0])
print(logits.shape)
output = F.softmax()
print(output)

torch.Size([100, 100])


YOUR ANSWER HERE

**(Question)** Implement a simple training loop to overfit on a small input sequence. The loss function should be a categorical cross entropy on the predicted characters. Monitor the loss function value over the iterations.

In [96]:
# Sample a small input sequence into tensor `input_seq` and store its corresponding expected sequence into tensor `target_seq`
# YOUR CODE HERE
vocab_size = len(ctoi)
embedding_dim = 8
hidden_dim = 30
input_seq = text_to_tensor(text_data[:100], ctoi)

id = torch.eye(len(ctoi))
one_hot_input = id[input_seq]
print(one_hot_input.shape)

target_seq = torch.cat([input_seq[1:], input_seq[:1]])
one_hot_target = id[target_seq]
print(one_hot_target.shape)
criterion = nn.CrossEntropyLoss()

print(input_seq.shape, target_seq.shape)

# Implement a training loop overfitting an input sequence and monitoring the loss function
def train_overfit(model, input_seq, target_seq, n_iters=200, learning_rate=0.2):
    # YOUR CODE HERE
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    hidden = None

    for iter in range(1, n_iters + 1):
      logits, hidden = model.forward(input_seq)
      output = F.softmax(logits, dim=1)
      loss = criterion(output, target_seq)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if iter % 10 == 0:
          print(f'Iteration {iter}/{n_iters}, Loss: {loss.item()}')



# Initialize a model and make it overfit the input sequence
# YOUR CODE HERE

charRNN = CharRNN(vocab_size, embedding_dim, hidden_dim)
train_overfit(charRNN, input_seq, one_hot_target)

torch.Size([100, 71])
torch.Size([100, 71])
torch.Size([100]) torch.Size([100])
Iteration 10/200, Loss: 4.261947154998779
Iteration 20/200, Loss: 4.261857509613037
Iteration 30/200, Loss: 4.261765956878662
Iteration 40/200, Loss: 4.2616729736328125
Iteration 50/200, Loss: 4.261578559875488
Iteration 60/200, Loss: 4.261481285095215
Iteration 70/200, Loss: 4.261382579803467
Iteration 80/200, Loss: 4.2612810134887695
Iteration 90/200, Loss: 4.261176586151123
Iteration 100/200, Loss: 4.261070728302002
Iteration 110/200, Loss: 4.260961532592773
Iteration 120/200, Loss: 4.260849475860596
Iteration 130/200, Loss: 4.260733604431152
Iteration 140/200, Loss: 4.260614395141602
Iteration 150/200, Loss: 4.260491847991943
Iteration 160/200, Loss: 4.260364532470703
Iteration 170/200, Loss: 4.260232925415039
Iteration 180/200, Loss: 4.260096549987793
Iteration 190/200, Loss: 4.259955406188965
Iteration 200/200, Loss: 4.259807109832764


**(Question)** Implement a `predict_argmax` method for your `RNN` model. Then, verify your overfitting: use some characters of your input sequence as context to predict the remaining ones. Experiment with the current model and analyze the results.

In [105]:
class CharRNN(CharRNN):
    def predict_argmax(self, context_tensor, n_predictions):
        # Apply the forward pass for the context tensor
        # Then, store the last prediction and last hidden state
        # YOUR CODE HERE
        predictions = []
        logits, hidden = self.forward(context_tensor)
        output = F.softmax(logits, dim=1)
        pred = torch.argmax(output)
        predictions.append(pred)
        # Use the last prediction and last hidden state as inputs to the next forward pass
        # Do this in a loop to predict the next `n_predictions` characters
        # YOUR CODE HERE
        for _ in range(n_predictions):
            logits, hidden = self.forward(pred, hidden)
            output = F.softmax(logits, dim=1)
            pred = torch.argmax(output)
            predictions.append(pred)
        
        return predictions


# Initialize a model and make it overfit as above
# Then, verify your overfitting by predicting characters given some context
# YOUR CODE HERE

charRNN = CharRNN(vocab_size, embedding_dim, hidden_dim)
train_overfit(charRNN, input_seq, one_hot_target)

print(charRNN.predict_argmax(text_to_tensor(text_data[:2],ctoi),10))


Iteration 10/200, Loss: 4.2622971534729
Iteration 20/200, Loss: 4.262202739715576
Iteration 30/200, Loss: 4.262106418609619
Iteration 40/200, Loss: 4.262007713317871
Iteration 50/200, Loss: 4.261906623840332
Iteration 60/200, Loss: 4.261804103851318
Iteration 70/200, Loss: 4.2616987228393555
Iteration 80/200, Loss: 4.261590480804443
Iteration 90/200, Loss: 4.261480331420898
Iteration 100/200, Loss: 4.261367321014404
Iteration 110/200, Loss: 4.261251926422119
Iteration 120/200, Loss: 4.261132717132568
Iteration 130/200, Loss: 4.261010646820068
Iteration 140/200, Loss: 4.260885715484619
Iteration 150/200, Loss: 4.260756492614746
Iteration 160/200, Loss: 4.260623931884766
Iteration 170/200, Loss: 4.2604875564575195
Iteration 180/200, Loss: 4.26034688949585
Iteration 190/200, Loss: 4.260201454162598
Iteration 200/200, Loss: 4.260051250457764


ValueError: RNN: Expected input to be 2D or 3D, got 1D tensor instead

YOUR ANSWER HERE

Using the argmax function to predict the next character can yield a deterministic generator always predicting the same characters. Instead, it is common to predict the next character by sampling from the distribution of output predictions, adding some randomness into the generator.

**(Question)** Implement a `predict_proba` method for your `RNN` model. It should be very similar to `predict_argmax`, but instead of using argmax, it should randomly sample from the output predictions. To do that, you can use the `torch.distribution.Categorical` class and its `sample()` method. Verify that your method correctly added some randomness.

In [ ]:
class CharRNN(CharRNN):
    def predict_proba(self, input_context, n_predictions):
        # YOUR CODE HERE
        raise NotImplementedError()

# Verify that your predictions are not deterministic anymore
# YOUR CODE HERE
raise NotImplementedError()

## 3. Train the RNN model on text data

**(Question)** Adapt your previous code to implement a proper training loop for a text dataset. To do so, we need to specify a sequence length `seq_len`, acting similarly to the batch size in classic neural networks. Then, you can either randomly sample sequences of length `seq_len` from the text dataset over `n_iters` iterations, or properly loop over the text dataset for `n_epochs` epochs (with a random starting point for each epoch to ensure different sequences), to make sure the whole dataset is seen by the model. Feel free to adjust training and model parameters empirically. Start with a small model and a small subset of the text dataset, then move on to larger experiments. Remember to use GPU if available.

In [ ]:
# Create the text dataset, compute its mappings and convert it to tensor
# YOUR CODE HERE
raise NotImplementedError()

# Initialize training parameters
# YOUR CODE HERE
raise NotImplementedError()

# Initialize a character-level RNN model
# YOUR CODE HERE
raise NotImplementedError()

# Setup the training loop
# Regularly record the loss and sample from the model to monitor what is happening
# YOUR CODE HERE
raise NotImplementedError()

**(Question)** From your trained model, play around with its predictions: start with a custom input sequence and ask the model to predict the rest. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

## 4. Experiment with different RNN architectures

**(Question)** Experiment with different RNN architecures. Potential ideas are multi-layer RNNs, GRUs and LSTMs. All models can be extended to multi-layer using the `num_layers` parameter. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE